# Load Libraries

In [1]:
import os
import warnings
import logging
import sys
import pickle
import numpy as np
import pandas as pd
import geopandas as gpd
import dotenv
import pyet
import matplotlib.pyplot as plt

dotenv.load_dotenv()
logging.basicConfig(level=logging.INFO)
warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', '{:.2f}'.format)
pd.set_option('display.max_colwidth', None)

# Load Data

In [2]:
FOLDER_PATH = "D:\\Pooya\\Storegs\\Dropbox\\IRIMO\Export\\"
monthly_data = pd.read_parquet(path=FOLDER_PATH + "Iran_Monthly_ETo_1951_2025.parquet")
monthly_daily_data = pd.read_parquet(path=FOLDER_PATH + "Iran_Monthly_Daily_ETo_1951_2025.parquet")

# Filter Data

In [3]:
selected_region = "Mazandaran"

monthly_data_selected = monthly_data.query("region_name == @selected_region")\
    .sort_values(by=["region_name", "station_name", "station_id", "year", "month"])\
        .reset_index(drop=True)
monthly_daily_data_selected = monthly_daily_data.query("region_name == @selected_region")\
    .sort_values(by=["region_name", "station_name", "station_id", "year", "month"])\
        .reset_index(drop=True)

In [4]:
monthly_data_selected

,year,month,region_id,region_name,station_id,station_name,lat,lon,station_elevation,tmax,tmax_count,tmin,tmin_count,tm,tm_count,umax,umax_count,umin,umin_count,um,um_count,ffm,ffm_count,sshn,sshn_count,pm,pm_count,p0m,p0m_count,ewm,ewm_count,radglo24,radglo24_count,td_m,td_m_count,twet_m,twet_m_count,tsoil_m,tsoil_m_count,ewsm,ewsm_count,evt,evt_count,rrr24,rrr24_count,date,Penman,Penman-Monteith,ASCE-PM,FAO-56,Priestley-Taylor,Kimberly-Penman,Thom-Oliver,Blaney-Criddle,Hamon,Romanenko,Linacre,Turc,Jensen-Haise,Mcguinness-Bordne,Hargreaves,FAO-24,Abtew,Makkink,Oudin
0,1951,1,MASA,Mazandaran,99361,Alasht,36.07,52.84,1805.00,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,0.00,0,0.00,0,1951-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1951,2,MASA,Mazandaran,99361,Alasht,36.07,52.84,1805.00,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,0.00,0,0.00,0,1951-02-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1951,3,MASA,Mazandaran,99361,Alasht,36.07,52.84,1805.00,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,0.00,0,0.00,0,1951-03-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1951,4,MASA,Mazandaran,99361,Alasht,36.07,52.84,1805.00,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,0.00,0,0.00,0,1951-04-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1951,5,MASA,Mazandaran,99361,Alasht,36.07,52.84,1805.00,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,0.00,0,0.00,0,1951-05-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21523,2025,5,MASA,Mazandaran,18332,Tonekabon,36.81,50.90,0.00,23.29,31,16.94,31,20.11,31,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,0.00,0,22.40,31,2025-05-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,116.01,147.71,NaN,115.04,NaN,NaN,186.16,110.81,NaN,NaN,NaN,126.64
21524,2025,6,MASA,Mazandaran,18332,Tonekabon,36.81,50.90,0.00,28.14,30,20.48,30,24.31,30,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,0.00,0,20.80,30,2025-06-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,136.62,200.37,NaN,151.32,NaN,NaN,220.67,137.29,NaN,NaN,NaN,150.12
21525,2025,7,MASA,Mazandaran,18332,Tonekabon,36.81,50.90,0.00,31.49,25,25.30,27,28.06,24,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,0.00,0,2.60,30,2025-07-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,154.81,253.62,NaN,194.60,NaN,NaN,252.55,136.38,NaN,NaN,NaN,171.81
21526,2025,8,MASA,Mazandaran,18332,Tonekabon,36.81,50.90,0.00,31.90,31,24.66,31,28.28,31,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,0.00,0,48.50,30,2025-08-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,143.21,226.39,NaN,201.33,NaN,NaN,231.71,135.07,NaN,NaN,NaN,157.63


In [5]:
monthly_daily_data_selected

,region_id,region_name,station_id,station_name,year,month,lat,lon,station_elevation,tmax,tmax_percent,tmin,tmin_percent,tm,tm_percent,umax,umax_percent,umin,umin_percent,um,um_percent,ffm,ffm_percent,sshn,sshn_percent,rrr24,rrr24_percent,pm,pm_percent,p0m,p0m_percent,ewm,ewm_percent,radglo24,radglo24_percent,evt,evt_percent,td_m,td_m_percent,twet_m,twet_m_percent,tsoil_m,tsoil_m_percent,ewsm,ewsm_percent,Penman,Penman_percent,Penman-Monteith,Penman-Monteith_percent,ASCE-PM,ASCE-PM_percent,FAO-56,FAO-56_percent,Priestley-Taylor,Priestley-Taylor_percent,Kimberly-Penman,Kimberly-Penman_percent,Thom-Oliver,Thom-Oliver_percent,Blaney-Criddle,Blaney-Criddle_percent,Hamon,Hamon_percent,Romanenko,Romanenko_percent,Linacre,Linacre_percent,Turc,Turc_percent,Jensen-Haise,Jensen-Haise_percent,Mcguinness-Bordne,Mcguinness-Bordne_percent,Hargreaves,Hargreaves_percent,FAO-24,FAO-24_percent,Abtew,Abtew_percent,Makkink,Makkink_percent,Oudin,Oudin_percent
0,MASA,Mazandaran,99361,Alasht,1951,1,36.07,52.84,1805.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00
1,MASA,Mazandaran,99361,Alasht,1951,2,36.07,52.84,1805.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00
2,MASA,Mazandaran,99361,Alasht,1951,3,36.07,52.84,1805.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00
3,MASA,Mazandaran,99361,Alasht,1951,4,36.07,52.84,1805.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00
4,MASA,Mazandaran,99361,Alasht,1951,5,36.07,52.84,1805.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21523,MASA,Mazandaran,18332,Tonekabon,2025,5,36.81,50.90,0.00,23.29,100.00,16.94,100.00,20.11,100.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,22.40,100.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,116.19,100.00,149.31,100.00,NaN,0.00,116.05,100.00,NaN,0.00,NaN,0.00,186.37,100.00,110.40,100.00,NaN,0.00,NaN,0.00,NaN,0.00,126.79,100.00
21524,MASA,Mazandaran,18332,Tonekabon,2025,6,36.81,50.90,0.00,28.14,100.00,20.48,100.00,24.31,100.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,20.80,100.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,136.25,100.00